This notebook takes a list of variants of interest and retrieves the list of participants who are heterozygous for each specific variant using Hail.

Inputs: 
- **mt_path:** Path to the input file containing genotype data. In this example, we use the All of Us ACAf smaller callset split Hail MatrixTable.


- **bed_path:** Path to the input file containing information about variants of interest. This file should include at least the locus and allele information for the variants and is expected to have a header. Locus is enough if you are not expecting exact match.

We recommend saving you bed file as a hail table to speedup the analysis

<div class="alert alert-block alert-info">
    
    
<li> Main node: 4CPUs, 15GB RAM, 100 GB Disk</li>

<li> Workers (2/0): 4CPUs, 15GB RAM, 150GB Disk</li>

<li> Time and Cost: \$0.73/h, ~5min, \$0.048</li>

</div>

# Setup

In [ ]:
import hail as hl
hl.default_reference("GRCh38")

In [ ]:
import os
bucket = os.getenv("WORKSPACE_BUCKET")
bucket

In [ ]:
from datetime import datetime
start = datetime.now()
start

# Inputs

- **mt_path:** Path to the input file containing genotype data. In this example, we use the All of Us ACAf smaller callset split Hail MatrixTable.


- **bed_path:** Path to the input file containing information about variants of interest. This file should include at least the locus and allele information for the variants and is expected to have a header. Locus is enough if you are not expecting exact match.

In [ ]:
# path to inputs
mt_path = "gs://fc-aou-datasets-controlled/v8/wgs/short_read/snpindel/acaf_threshold/splitMT/hail.mt"
bed_path = "gs://fc-secure-ff68a895-e88d-426e-9ae4-b3802c51b53b/notebooks/data/random_78_sites_GRCh38.ht"

In [ ]:
# load bed fiel into Hail
bed = hl.read_table(bed_path)
bed = bed.key_by("locus", "alleles")
bed.show(5)

In [ ]:
# load matrixtable into Hail
mt = hl.read_matrix_table(mt_path)
mt.count()

# Filter samples

We filter for African ancestry to avoid encountering resource issues while loading the output file. It takes the same time to work on all participants.

In [ ]:
#############################remove in real analysis
ancestry_pred_path = "gs://fc-aou-datasets-controlled/v8/wgs/short_read/snpindel/aux/ancestry/ancestry_preds.tsv"
def filter_by_ancestry(mt, ancestry_pred_path, ancestry_filter="eur"):
    """
    Imports ancestry prediction data and filters a Hail MatrixTable by a specified ancestry.

    Parameters:
        mt (hl.MatrixTable): The input MatrixTable.
        ancestry_pred_path (str): Path to the ancestry prediction file. This function use the all of us ancestry file, please make sure the format is the same as the all of us ancestry file if you are using other pop files.
        ancestry_filter (str): The ancestry label to filter on (default is "eur").

    Returns:
        hl.MatrixTable: Filtered MatrixTable with ancestry annotations.
    """
    ancestry_pred = hl.import_table(
        ancestry_pred_path,
        key="research_id",
        impute=True,
        types={"research_id": hl.tstr, "pca_features": hl.tarray(hl.tfloat)}
    )
    
    mt = mt.annotate_cols(ancestry_pred=ancestry_pred[mt.s])
    mt = mt.filter_cols(mt.ancestry_pred.ancestry_pred_other == ancestry_filter)
    
    return mt

# Usage
#ancestry_pred_path = "gs://fc-aou-preprod-datasets-controlled/v8/wgs/short_read/snpindel/aux/ancestry/echo_v4_r2.ancestry_preds.tsv"
#mt = filter_by_ancestry(mt, ancestry_pred_path, ancestry_filter="eur")
mt =  filter_by_ancestry(mt, ancestry_pred_path, ancestry_filter="afr")
mt.count()
###################################remove in real analysis

# Filter variants

Filter the MatrixTable based on the intervals in the input BED file, and optionally, match the alleles as well.

In [ ]:
# filter to test interval or variants in bed file
def filter_mt_by_intervals(mt, intervals):
    """
    Filters a Hail MatrixTable to retain only specified genomic intervals.

    Parameters:
        mt (hl.MatrixTable): The input MatrixTable.
        intervals (list): A list of genomic intervals in string format (e.g., 'chr5:66715539-66715540').
                            examples: test_intervals = ['chr1:100M-200M', 'chr16:29.01M-29.02M']
                                        test_intervals = ['chr21']

    Returns:
        hl.MatrixTable: Filtered MatrixTable.
    """
    parsed_intervals = [hl.parse_locus_interval(x) for x in intervals]
    return hl.filter_intervals(mt, parsed_intervals)

# Usage

#test_intervals = [
#    'chr5:66715539-66715540', 
 #   'chrX:9843055-9843056', 
  #  'chrX:51502424-51502425', 
   # 'chrX:67872303-67872304'
#]

#mt = filter_mt_by_intervals(mt, test_intervals)
#mt.row.show()

In [ ]:
# create intervals and filter MT by intervals
bed = bed.annotate(interval = bed.chr_grch38 + ":" + hl.str(bed.pos_grch38) + "-" + hl.str(bed.pos_grch38 + 1))
test_interval = bed.interval.collect()
mt = filter_mt_by_intervals(mt, test_interval)
mt.count()

In [ ]:
#annotatemt by bed file in case there are other information of interest in the bed file 
mt = mt.annotate_rows(bed=bed[mt.row_key])
mt.row.show(5)

# Exact match filtering

Filtering by intervals will retain variants with alleles not present in the bed file. We can filter the rows to remove these extra variants, but keep in mind that row filtering can be slow and expensive. If this step doesn't affect your downstream analysis, you can skip it.

In [ ]:
mt_filtered = mt.filter_rows(~hl.is_missing(mt.bed.chr_grch38))
mt_filtered.count()

# Filter genotypes

In [ ]:
# Filter for heterozygous genotypes, others will be removed from the MT
het_variants = mt_filtered.filter_entries(mt_filtered.GT.is_het())

# Extract information needed

In [ ]:
# Convert MatrixTable to a Table with heterozygous samples
result_table = het_variants.entries()
result_table = result_table.group_by(
    locus = result_table.locus,
    alleles = result_table.alleles
).aggregate(
    heterozygous_samples = hl.agg.collect_as_set(result_table.s)
)

In [ ]:
# check first several rows of the results
result_table.show(5)

# Save results to bucket

**We recommend load the saved file into Hail or other tools in a new notebook to avoid encountering any resource issues.**

## Save results as a tsv file

In [ ]:
result_table.export(f'{bucket}/data/heterozygous_participants.tsv')

In [ ]:
# load the saved tsv into pandas
import pandas as pd
het_samples = f'{bucket}/data/heterozygous_participants.tsv'
het_samples = pd.read_csv(het_samples, sep = "\t")
print(het_samples.head(5))

# Convert string representation of lists to actual Python lists
import ast
het_samples["heterozygous_samples"] = het_samples["heterozygous_samples"].apply(ast.literal_eval)
print(f"{len(het_samples['heterozygous_samples'][0])} heterozygotes for the variant at locus {het_samples['locus'][0]} with alleles {het_samples['alleles'][0]}")

## save results to buckst as a hail table

In [ ]:
result_table.write(f'{bucket}/data/heterozygous_participants.ht')

In [ ]:
# load saved file into Hail
het_samples_ht = f'{bucket}/data/heterozygous_participants.ht'
het_samples_ht = hl.read_table(het_samples_ht)
het_samples_ht.show(5)

In [ ]:
stop = datetime.now()
stop

In [ ]:
str(stop-start)

In [ ]:
!gsutil -m ls gs://fc-secure-ff68a895-e88d-426e-9ae4-b3802c51b53b/data/